In [5]:
import torch
import time
import scipy
from memory_profiler import memory_usage
import cProfile
import pstats

from torchfem import Solid
from torchfem.materials import Isotropic

In [2]:
def get_cube(N):
    # Create nodes
    grid = torch.linspace(0, 1, N)
    x, y, z = torch.meshgrid(grid, grid, grid, indexing="ij")
    nodes = torch.vstack([x.ravel(), y.ravel(), z.ravel()]).T

    # Create elements
    indices = torch.arange(N**3).reshape((N, N, N))
    n0 = indices[:-1, :-1, :-1].ravel()
    n1 = indices[1:, :-1, :-1].ravel()
    n2 = indices[:-1, 1:, :-1].ravel()
    n3 = indices[1:, 1:, :-1].ravel()
    n4 = indices[:-1, :-1, 1:].ravel()
    n5 = indices[1:, :-1, 1:].ravel()
    n6 = indices[:-1, 1:, 1:].ravel()
    n7 = indices[1:, 1:, 1:].ravel()
    elements = torch.vstack([n0, n1, n3, n2, n4, n5, n7, n6]).T

    # Material model
    material = Isotropic(E=1000.0, nu=0.3)

    # Assign boundary conditions
    forces = torch.zeros_like(nodes, requires_grad=True)
    constraints = torch.zeros_like(nodes, dtype=bool)
    constraints[nodes[:, 0] == 0.0, :] = True
    constraints[nodes[:, 0] == 1.0, 0] = True
    displacements = torch.zeros_like(nodes)
    displacements[nodes[:, 0] == 1.0, 0] = 0.1

    return Solid(nodes, elements, forces, displacements, constraints, material.C())

In [ ]:
results = {}
for N in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
    print(f"Running N={N}")
    box = get_cube(N)
    dofs = box.n_dofs

    # Forward pass
    start_time = time.time()
    mem_usage, (u, f) = memory_usage(lambda: box.solve(), retval=True, interval=0.1)
    end_time = time.time()
    fwd_mem_usage = max(mem_usage) - min(mem_usage)
    fwd_time = end_time - start_time
    print(f"  ... forward pass with {dofs} DOFs done in {fwd_time:.2f}s.")

    # Backward pass
    start_time = time.time()
    mem_usage = memory_usage(lambda: u.sum().backward(retain_graph=True), interval=0.1)
    end_time = time.time()
    bwd_mem_usage = max(mem_usage) - min(mem_usage)
    bwd_time = end_time - start_time
    print(f"  ... backward pass with {dofs} DOFs done in {bwd_time:.2f}.")

    results[N] = (
        dofs,
        fwd_time,
        fwd_mem_usage,
        bwd_time,
        bwd_mem_usage,
    )

In [ ]:
# cProfile
cProfile.run("get_cube(70).solve()", "stats")

In [ ]:
p = pstats.Stats("stats")
p.sort_stats("tottime").print_stats(10)

In [ ]:
# Format results as a table
print("|  N  |    DOFs | FWD Time |  FWD Memory | BWD Time |  BWD Memory |")
print("| --- | ------- | -------- | ----------- | -------- | ----------- |")
for N, (dofs, fwd_t, fwd_mem, bwd_t, bwd_mem) in results.items():
    print(
        f"| {N:3d} | {dofs:7d} |"
        f" {fwd_t:7.2f}s |  {fwd_mem:7.2f} MB |"
        f" {bwd_t:7.2f}s |  {bwd_mem:7.2f} MB |"
    )


scipy.show_config()